## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "Charles Schwab Challenge"
tournament_date = "5/25/2025"  # Ending date of tournament
old_course = "Colonial Country Club"
tournament_id = "R2025021"

# New Tournament
new_tournament_name = "the Memorial Tournament presented by Workday"
new_ending_date = "6/1/2025"
new_course = "Muirfield Village Golf Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025021 (Charles Schwab Challenge), year: 20250
✅ 135 new rows added for 'Charles Schwab Challenge'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
11,2025,2025-05-25,R2025021,Charles Schwab Challenge,Colonial Country Club,Ben Griffin,1,1,-4,-7,-2,+1,"$1,710,000.00",500.000
86,2025,2025-05-25,R2025021,Charles Schwab Challenge,Colonial Country Club,Matti Schmid,2,2,-4,-7,-2,+2,"$1,035,500.00",300.000
19,2025,2025-05-25,R2025021,Charles Schwab Challenge,Colonial Country Club,Bud Cauley,3,3,-4,-1,-1,-3,"$655,500.00",190.000
126,2025,2025-05-25,R2025021,Charles Schwab Challenge,Colonial Country Club,Tommy Fleetwood,T4,4,-3,-3,E,-2,"$427,500.00",122.500
116,2025,2025-05-25,R2025021,Charles Schwab Challenge,Colonial Country Club,Scottie Scheffler,T4,4,-2,+1,-6,-1,"$427,500.00",122.500


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,580,.2011,2025
1,Aaron Baddeley,167.0,-0.965,178.0,-1.366,137.0,-0.203,3.0,0.604,161.0,...,288.9,119.0,56.64%,171.0,60.61%,89.0,60.68%,423,.3101,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360,.3777,2025
3,Aaron Rai,19.0,0.867,26.0,0.355,19.0,0.523,104.0,-0.011,119.0,...,287.3,1.0,73.36%,7.0,70.05%,129.0,58.87%,30,2.7271,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,770,.1267,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [2]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 9 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
608,2016,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2016-06-05
489,2017,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2017-06-04
369,2018,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2018-06-03
249,2019,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2019-06-02
119,2020,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2020-07-19
0,2021,Muirfield Village Golf Club,023,the Memorial Tournament presented by Nationwide,2021-06-06
848,2022,Muirfield Village Golf Club,023,the Memorial Tournament presented by Workday,2022-06-05
967,2023,Muirfield Village Golf Club,R2023023,the Memorial Tournament presented by Workday,2023-06-04
1087,2024,Muirfield Village Golf Club,R2024023,the Memorial Tournament presented by Workday,2024-06-09


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-06-05 — the Memorial Tournament presented by Nationwide (417 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,10,378.0,62.5,23.62,1,2016-06-05,the Memorial Tournament presented by Nationwide
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-06-05,the Memorial Tournament presented by Nationwide
2,Abraham Ancer,10,2,43.0,20.0,4.30,2,2016-06-05,the Memorial Tournament presented by Nationwide



📆 2017-06-04 — the Memorial Tournament presented by Nationwide (404 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,8,264.29,53.3,17.62,0,2017-06-04,the Memorial Tournament presented by Nationwide
1,Aaron Wise,6,4,0.00,66.7,0.00,4,2017-06-04,the Memorial Tournament presented by Nationwide
2,Abraham Ancer,1,1,0.00,100.0,0.00,1,2017-06-04,the Memorial Tournament presented by Nationwide



📆 2018-06-03 — the Memorial Tournament presented by Nationwide (408 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,5,2,56.96,40.0,11.39,0,2022-06-05,the Memorial Tournament presented by Workday
1,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-06-05,the Memorial Tournament presented by Workday
2,Aaron Jarvis,1,0,0.00,0.0,0.00,0,2022-06-05,the Memorial Tournament presented by Workday



📆 2023-06-04 — the Memorial Tournament presented by Workday (488 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,11,342.286,78.6,24.45,5,2023-06-04,the Memorial Tournament presented by Workday
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-06-04,the Memorial Tournament presented by Workday
2,Aaron Jarvis,1,0,0.000,0.0,0.00,0,2023-06-04,the Memorial Tournament presented by Workday



📆 2024-06-09 — the Memorial Tournament presented by Workday (468 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,9,181.975,64.3,13.00,0,2024-06-09,the Memorial Tournament presented by Workday
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-06-09,the Memorial Tournament presented by Workday
2,Aaron Rai,15,12,442.007,80.0,29.47,6,2024-06-09,the Memorial Tournament presented by Workday


### Recent Form

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-06-05 — the Memorial Tournament presented by Nationwide (417 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Lee McCoy,1,4.0,5.77,2016-06-05,the Memorial Tournament presented by Nationwide
1,Cody Gribble,1,8.0,11.54,2016-06-05,the Memorial Tournament presented by Nationwide
2,Jon Rahm,2,12.5,11.38,2016-06-05,the Memorial Tournament presented by Nationwide



📆 2017-06-04 — the Memorial Tournament presented by Nationwide (404 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Keith Mitchell,1,11.0,15.87,2017-06-04,the Memorial Tournament presented by Nationwide
1,Oscar Fraustro,1,13.0,18.76,2017-06-04,the Memorial Tournament presented by Nationwide
2,Cam Davis,1,15.0,21.64,2017-06-04,the Memorial Tournament presented by Nationwide



📆 2018-06-03 — the Memorial Tournament presented by Nationwide (408 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Taylor Montgomery,1,11.0,15.87,2022-06-05,the Memorial Tournament presented by Workday
1,Haotong Li,1,12.0,17.31,2022-06-05,the Memorial Tournament presented by Workday
2,Martin Contini,1,16.0,23.08,2022-06-05,the Memorial Tournament presented by Workday



📆 2023-06-04 — the Memorial Tournament presented by Workday (488 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-06-04,the Memorial Tournament presented by Workday
1,Brooks Koepka,2,1.5,1.37,2023-06-04,the Memorial Tournament presented by Workday
2,Scottie Scheffler,15,8.4,3.03,2023-06-04,the Memorial Tournament presented by Workday



📆 2024-06-09 — the Memorial Tournament presented by Workday (468 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,4.0,3.64,2024-06-09,the Memorial Tournament presented by Workday
1,Scottie Scheffler,13,4.5,1.71,2024-06-09,the Memorial Tournament presented by Workday
2,Dean Burmester,1,12.0,17.31,2024-06-09,the Memorial Tournament presented by Workday


### Course History

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for the Memorial Tournament presented by Nationwide on 2016-06-05


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Adam Hadwin,1,57.0,82.23,2016-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide
1,Alex Cejka,1,90.0,129.84,2016-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide
2,Andres Gonzales,1,90.0,129.84,2016-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide



🏌️‍♂️ Course history for the Memorial Tournament presented by Nationwide on 2017-06-04


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2017-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide
1,Adam Hadwin,2,34.0,30.95,2017-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide
2,Alex Cejka,2,73.5,66.90,2017-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Nationwide



🏌️‍♂️ Course history for the Memorial Tournament presented by Nationwide on 2018-06-03


...

,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,67.3,48.55,2022-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
1,Aaron Wise,3,63.0,45.44,2022-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
2,Abraham Ancer,3,60.0,43.28,2022-06-05,Muirfield Village Golf Club,the Memorial Tournament presented by Workday



🏌️‍♂️ Course history for the Memorial Tournament presented by Workday on 2023-06-04


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,67.3,48.55,2023-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
1,Aaron Rai,1,26.0,37.51,2023-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
2,Aaron Wise,4,47.8,29.70,2023-06-04,Muirfield Village Golf Club,the Memorial Tournament presented by Workday



🏌️‍♂️ Course history for the Memorial Tournament presented by Workday on 2024-06-09


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,22.0,31.74,2024-06-09,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
1,Aaron Rai,2,58.0,52.79,2024-06-09,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
2,Aaron Wise,4,47.8,29.70,2024-06-09,Muirfield Village Golf Club,the Memorial Tournament presented by Workday


## Training Dataset

In [6]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Aaron Baddeley,CUT,90,146.0,-0.330,159.0,-0.366,164.0,-0.362,10.0,0.402,8.0,0.586,59.0,20.72,78.0,3.06,14.0,4.01,50.0,4.64,159.0,237.0,82.0,291.7,155.0,55.42,154.0,63.11,5.0,64.3,143.0,1.14,NaN,62.5,378.00,23.62,1.0,54.1,19.09,NaN,NaN,0
1,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Adam Hadwin,T11,11,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,NaN,72.2,426.67,23.70,5.0,51.9,17.63,57.0,82.23,1
2,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Alex Cejka,T57,57,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,NaN,57.1,491.17,35.08,0.0,52.5,19.39,90.0,129.84,0
3,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Andrew Loupe,CUT,90,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,NaN,58.8,482.82,28.40,0.0,56.0,19.37,NaN,NaN,0
4,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Anirban Lahiri,T61,61,97.0,0.088,95.0,0.025,90.0,0.143,130.0,-0.080,99.0,-0.020,107.0,19.73,136.0,3.09,97.0,4.05,155.0,4.74,154.0,232.0,82.0,291.7,150.0,56.31,161.0,62.88,138.0,56.4,92.0,1.63,100.0,70.0,269.75,26.98,4.0,50.5,21.06,NaN,NaN,0
5,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Ben Martin,T38,38,126.0,-0.117,79.0,0.079,101.0,0.081,169.0,-0.277,70.0,0.140,29.0,21.53,146.0,3.10,97.0,4.05,28.0,4.61,71.0,174.0,108.0,288.8,66.0,62.14,64.0,66.73,160.0,54.84,123.0,1.27,NaN,66.7,297.05,19.80,1.0,56.9,20.52,90.0,129.84,0
6,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Bill Haas,CUT,90,34.0,0.693,81.0,0.068,54.0,0.272,14.0,0.353,130.0,-0.174,172.0,17.89,22.0,3.02,68.0,4.04,100.0,4.69,55.0,159.0,97.0,289.6,62.0,62.34,57.0,67.14,12.0,63.3,44.0,2.87,60.0,78.6,932.50,66.61,2.0,36.2,13.37,18.0,25.97,0
7,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Brendan Steele,T20,20,50.0,0.542,39.0,0.331,63.0,0.238,161.0,-0.188,153.0,-0.287,82.0,20.17,136.0,3.09,114.0,4.06,13.0,4.58,15.0,114.0,20.0,302.2,94.0,60.29,25.0,68.72,162.0,54.76,88.0,1.69,150.0,73.3,602.08,40.14,3.0,44.9,16.19,63.0,90.89,1
8,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Brendon Todd,CUT,90,184.0,-2.055,184.0,-1.043,184.0,-1.424,5.0,0.412,17.0,0.462,183.0,15.78,184.0,3.21,177.0,4.11,164.0,4.75,178.0,292.0,167.0,280.1,125.0,58.02,185.0,54.23,35.0,61.46,364.0,0.51,NaN,20.0,242.73,12.14,0.0,81.4,26.74,17.0,24.53,0
9,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Brendon de Jonge,CUT,90,163.0,-0.488,143.0,-0.229,145.0,-0.195,125.0,-0.063,63.0,0.156,156.0,18.59,102.0,3.07,136.0,4.07,68.0,4.66,153.0,231.0,122.0,287.5,109.0,59.19,93.0,65.64,136.0,56.49,429.0,0.44,NaN,42.1,322.67,16.98,0.0,69.6,23.23,90.0,129.84,0


In [7]:
# Play with different years to see if I notice anything wrong
training_df[training_df["SEASON"] == 2022].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
728,2022,2022-06-05,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Aaron Rai,T26,26,46.0,0.490,69.0,0.146,46.0,0.301,93.0,0.042,139.0,-0.106,151.0,20.04,125.0,3.07,76.0,4.02,44.0,4.58,42.0,151.0,135.0,295.4,16.0,67.37,23.0,68.79,75.0,60.2,173.0,1.00,150.0,68.4,317.41,16.71,2.0,56.6,18.89,NaN,NaN,0
729,2022,2022-06-05,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Aaron Wise,2,2,25.0,0.952,31.0,0.387,21.0,0.489,85.0,0.076,81.0,0.100,22.0,23.39,4.0,2.98,25.0,3.99,24.0,4.55,44.0,154.0,39.0,308.3,115.0,59.15,21.0,69.06,31.0,62.63,37.0,2.69,60.0,75.0,529.33,33.08,4.0,42.2,14.89,63.0,45.44,1
730,2022,2022-06-05,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Abraham Ancer,T32,32,122.0,-0.026,29.0,0.395,109.0,0.026,189.0,-0.447,62.0,0.165,61.0,21.84,62.0,3.04,25.0,3.99,81.0,4.62,18.0,124.0,120.0,297.1,4.0,70.54,77.0,66.86,64.0,60.69,24.0,3.21,60.0,66.7,313.03,20.87,0.0,51.2,18.47,60.0,43.28,0
731,2022,2022-06-05,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Adam Hadwin,T18,18,43.0,0.550,130.0,-0.084,49.0,0.274,14.0,0.361,73.0,0.127,81.0,21.54,31.0,3.02,58.0,4.01,64.0,4.60,49.0,161.0,117.0,297.2,44.0,63.71,52.0,67.84,46.0,61.61,82.0,1.60,100.0,76.5,531.52,31.27,1.0,44.0,15.22,61.1,29.38,1
732,2022,2022-06-05,the Memorial Tournament presented by Workday,Muirfield Village Golf Club,Adam Long,CUT,90,91.0,0.150,83.0,0.088,100.0,0.062,112.0,0.000,24.0,0.402,134.0,20.45,31.0,3.02,25.0,3.99,157.0,4.69,102.0,198.0,165.0,291.6,33.0,64.74,66.0,67.17,17.0,64.23,221.0,0.84,125.0,65.0,391.54,19.58,1.0,52.4,17.21,68.7,49.56,0


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [8]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Aaron Baddeley,CUT,90,146.0,-0.330,159.0,-0.366,164.0,-0.362,10.0,0.402,8.0,0.586,59.0,20.72,78.0,3.06,14.0,4.01,50.0,4.64,159.0,237.0,82.0,291.7,155.0,55.42,154.0,63.11,5.0,64.3,143.0,1.14,1000.0,62.5,378.00,23.62,1.0,54.1,19.09,NaN,NaN,0
1,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Adam Hadwin,T11,11,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,1000.0,72.2,426.67,23.70,5.0,51.9,17.63,57.0,82.23,1
2,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Alex Cejka,T57,57,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,1000.0,57.1,491.17,35.08,0.0,52.5,19.39,90.0,129.84,0
3,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Andrew Loupe,CUT,90,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,1000.0,58.8,482.82,28.40,0.0,56.0,19.37,NaN,NaN,0
4,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Anirban Lahiri,T61,61,97.0,0.088,95.0,0.025,90.0,0.143,130.0,-0.080,99.0,-0.020,107.0,19.73,136.0,3.09,97.0,4.05,155.0,4.74,154.0,232.0,82.0,291.7,150.0,56.31,161.0,62.88,138.0,56.4,92.0,1.63,100.0,70.0,269.75,26.98,4.0,50.5,21.06,NaN,NaN,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [9]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Aaron Baddeley,CUT,90,146.0,-0.330,159.0,-0.366,164.0,-0.362,10.0,0.402,8.0,0.586,59.0,20.72,78.0,3.06,14.0,4.01,50.0,4.64,159.0,237.0,82.0,291.7,155.0,55.42,154.0,63.11,5.0,64.30,143.0,1.14,1000.0,62.5,378.00,23.62,1.0,54.1,19.09,51.916169,49.737052,0
1,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Adam Hadwin,T11,11,111.0,-0.011,55.0,0.223,121.0,-0.055,132.0,-0.081,11.0,0.530,22.0,21.74,16.0,3.01,48.0,4.03,68.0,4.66,35.0,141.0,92.0,290.1,49.0,63.62,83.0,66.07,97.0,58.27,193.0,0.91,1000.0,72.2,426.67,23.70,5.0,51.9,17.63,57.000000,82.230000,1
2,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Alex Cejka,T57,57,39.0,0.631,148.0,-0.292,12.0,0.626,20.0,0.297,90.0,0.010,17.0,21.89,3.0,2.98,14.0,4.01,10.0,4.57,78.0,177.0,155.0,281.5,22.0,65.89,39.0,67.99,3.0,65.03,130.0,1.22,1000.0,57.1,491.17,35.08,0.0,52.5,19.39,90.000000,129.840000,0
3,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Andrew Loupe,CUT,90,150.0,-0.395,105.0,-0.005,154.0,-0.281,158.0,-0.172,38.0,0.280,12.0,22.74,78.0,3.06,177.0,4.11,13.0,4.58,101.0,189.0,5.0,309.3,184.0,45.56,148.0,63.41,175.0,52.83,194.0,0.91,1000.0,58.8,482.82,28.40,0.0,56.0,19.37,51.916169,49.737052,0
4,2016,2016-06-05,the Memorial Tournament presented by Nationwide,Muirfield Village Golf Club,Anirban Lahiri,T61,61,97.0,0.088,95.0,0.025,90.0,0.143,130.0,-0.080,99.0,-0.020,107.0,19.73,136.0,3.09,97.0,4.05,155.0,4.74,154.0,232.0,82.0,291.7,150.0,56.31,161.0,62.88,138.0,56.40,92.0,1.63,100.0,70.0,269.75,26.98,4.0,50.5,21.06,51.916169,49.737052,0


In [10]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [11]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [12]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [13]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,13300
1,Xander Schauffele,10900
2,Justin Thomas,10100
3,Collin Morikawa,9900
4,Patrick Cantlay,9500
5,Tommy Fleetwood,9400
6,Ludvig Aberg,9300
7,Corey Conners,9100
8,Hideki Matsuyama,9000
9,Viktor Hovland,8900


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [14]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,the Memorial Tournament presented by Workday,Scottie Scheffler,11/4,2.75
2,2025,the Memorial Tournament presented by Workday,Collin Morikawa,16/1,16.00
3,2025,the Memorial Tournament presented by Workday,Xander Schauffele,18/1,18.00
4,2025,the Memorial Tournament presented by Workday,Justin Thomas,20/1,20.00
5,2025,the Memorial Tournament presented by Workday,Patrick Cantlay,20/1,20.00
6,2025,the Memorial Tournament presented by Workday,Tommy Fleetwood,20/1,20.00
7,2025,the Memorial Tournament presented by Workday,Ludvig Aberg,25/1,25.00
8,2025,the Memorial Tournament presented by Workday,Viktor Hovland,30/1,30.00
9,2025,the Memorial Tournament presented by Workday,Corey Conners,30/1,30.00
10,2025,the Memorial Tournament presented by Workday,Hideki Matsuyama,35/1,35.00


## Cut Percentage and FedEx Points

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,55.951,46.7,3.73,0,2025-06-01,the Memorial Tournament presented by Workday
1,Aaron Rai,14,11,577.480,78.6,41.25,0,2025-06-01,the Memorial Tournament presented by Workday
2,Adam Hadwin,15,9,165.263,60.0,11.02,0,2025-06-01,the Memorial Tournament presented by Workday
3,Adam Long,1,0,0.000,0.0,0.00,0,2025-06-01,the Memorial Tournament presented by Workday
4,Adam Schenk,18,7,253.954,38.9,14.11,0,2025-06-01,the Memorial Tournament presented by Workday
5,Adam Scott,10,8,238.096,80.0,23.81,3,2025-06-01,the Memorial Tournament presented by Workday
6,Adam Svensson,19,11,173.158,57.9,9.11,1,2025-06-01,the Memorial Tournament presented by Workday
7,Adrien Dumont de Chassart,8,3,200.500,37.5,25.06,0,2025-06-01,the Memorial Tournament presented by Workday
8,Akshay Bhatia,14,11,731.490,78.6,52.25,1,2025-06-01,the Memorial Tournament presented by Workday
9,Aldrich Potgieter,11,4,441.000,36.4,40.09,1,2025-06-01,the Memorial Tournament presented by Workday


## Recent Form

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,3.5,3.19,2025-06-01,the Memorial Tournament presented by Workday
1,Scottie Scheffler,12,7.4,2.89,2025-06-01,the Memorial Tournament presented by Workday
2,Jon Rahm,2,11.0,10.01,2025-06-01,the Memorial Tournament presented by Workday
3,Rory McIlroy,8,11.8,5.37,2025-06-01,the Memorial Tournament presented by Workday
4,John Keefer,1,13.0,18.76,2025-06-01,the Memorial Tournament presented by Workday
5,Bubba Watson,1,14.0,20.20,2025-06-01,the Memorial Tournament presented by Workday
6,Taisei Shimizu,1,16.0,23.08,2025-06-01,the Memorial Tournament presented by Workday
7,Ian Gilligan,1,16.0,23.08,2025-06-01,the Memorial Tournament presented by Workday
8,Joaquin Niemann,2,18.5,16.84,2025-06-01,the Memorial Tournament presented by Workday
9,Tommy Fleetwood,11,18.8,7.57,2025-06-01,the Memorial Tournament presented by Workday


## Course History

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,22.0,31.74,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
1,Aaron Rai,2,58.0,52.79,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
2,Aaron Wise,4,47.8,29.70,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
3,Abraham Ancer,4,53.0,32.93,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
4,Adam Hadwin,7,54.4,26.16,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
5,Adam Long,5,77.2,43.09,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
6,Adam Schenk,4,47.0,29.20,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
7,Adam Scott,5,25.8,14.40,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
8,Adam Svensson,3,54.0,38.95,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday
9,Akshay Bhatia,2,56.0,50.97,2025-06-01,Muirfield Village Golf Club,the Memorial Tournament presented by Workday


## Merged Dataframe

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import load_all_stats  # New helper function

# === Load current season stats directly from database ===
season = tournament_config["new"]["season"]
stats_df = load_all_stats(db_path)
stats_df = stats_df[stats_df["SEASON"] == season].copy()

# === Standardize PLAYER columns for consistent merges ===
for df in [stats_df, odds_current, cuts_current, recent_form_current, course_hist_current, dk]:
    df["PLAYER"] = df["PLAYER"].astype(str).str.strip()

# === Merge all engineered and DK features ===
this_week = stats_df.copy()
this_week = this_week.merge(odds_current[["PLAYER", "VEGAS_ODDS"]], on="PLAYER", how="left")
this_week = this_week.merge(cuts_current[["PLAYER", "CUT_PERCENTAGE", "FEDEX_CUP_POINTS", "form_density", "CONSECUTIVE_CUTS"]], on="PLAYER", how="left")
this_week = this_week.merge(recent_form_current[["PLAYER", "RECENT_FORM", "adj_form"]], on="PLAYER", how="left")
this_week = this_week.merge(course_hist_current[["PLAYER", "COURSE_HISTORY", "adj_ch"]], on="PLAYER", how="left")
this_week = this_week.merge(dk[["PLAYER", "SALARY"]], on="PLAYER", how="right")  # keep only DK players

# === Final cleanup ===
this_week = this_week.sort_values("PLAYER").reset_index(drop=True)
this_week["SEASON"] = this_week["SEASON"].astype(float)  # allows averaging if needed

this_week.head()

,SEASON,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,2025.0,Aaron Rai,19.0,0.867,26.0,0.355,19.0,0.523,104.0,-0.011,119.0,-0.094,38.0,23.31%,176.0,3.17,5.0,3.95,29.0,4.54,79.0,171.0,170.0,287.3,1.0,73.36%,7.0,70.05%,129.0,58.87%,30.0,2.7271,50.0,78.6,577.480,41.25,0.0,36.6,13.52,58.0,52.79,7600
1,2025.0,Adam Hadwin,152.0,-0.564,144.0,-0.243,157.0,-0.480,51.0,0.159,114.0,-0.071,61.0,22.61%,30.0,3.02,126.0,4.05,139.0,4.65,117.0,196.0,105.0,299.1,91.0,59.00%,157.0,62.53%,164.0,55.17%,101.0,1.2731,300.0,60.0,165.263,11.02,0.0,64.0,23.08,54.4,26.16,6200
2,2025.0,Adam Scott,73.0,0.176,65.0,0.180,125.0,-0.148,56.0,0.144,90.0,0.051,67.0,22.53%,135.0,3.10,115.0,4.04,36.0,4.55,119.0,199.0,79.0,302.7,120.0,56.55%,112.0,64.51%,116.0,59.57%,41.0,2.3779,80.0,80.0,238.096,23.81,3.0,44.9,18.72,25.8,14.40,7200
3,2025.0,Akshay Bhatia,85.0,0.081,107.0,-0.015,35.0,0.387,154.0,-0.291,9.0,0.611,6.0,25.36%,7.0,2.97,32.0,3.99,162.0,4.69,141.0,218.0,157.0,293.3,61.0,60.87%,46.0,67.15%,158.0,56.25%,31.0,2.7051,80.0,78.6,731.490,52.25,1.0,38.1,14.07,56.0,50.97,7300
4,2025.0,Alex Noren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,72.0,1.7302,70.0,100.0,84.500,42.25,2.0,34.0,30.95,51.3,26.36,7400


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [19]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# === Normalize & Assign Odds for Poor or Unknown Players ===
this_week["VEGAS_ODDS"] = this_week["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === Normalize & Assign OWGR & OWGR_RANK for Unranked Players ===
# OWGR (raw) — assign worst actual rank in dataset
if "OWGR" in this_week.columns:
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).astype(float).clip(upper=1000)

# OWGR_RANK — if present, use 1000 for missing (very poor)
if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = this_week["OWGR_RANK"].fillna(1000).clip(upper=1000)

# === Assign RECENT FORM Score for Players Who Haven’t Played Recently ===
this_week["RECENT_FORM"] = this_week["RECENT_FORM"].fillna(90)

# === Assign 0 to FEDEX_CUP_POINTS Where No Data ===
this_week["FEDEX_CUP_POINTS"] = this_week["FEDEX_CUP_POINTS"].fillna(0)

# === Course History NaNs: Fill with Mean (not 0 — 0 implies bad, not missing) ===
if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(this_week["COURSE_HISTORY"].mean())

# === Fill Remaining NaNs in Numeric Columns with Column Mean ===
numeric_cols = this_week.select_dtypes(include=["float64", "int64"])
numeric_filled = numeric_cols.fillna(numeric_columns.mean())
this_week.update(numeric_filled)



In [20]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [21]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

## Correlations
**Last 4 Years**

In [22]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [23]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [24]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.910144   0.87904913 0.89109732 0.89235986 0.91659495]
Average ROC AUC score: 0.90


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [27]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Shane Lowry,8700,0.814180,29.9,40.8,35.00,81.8,14.0,1273.139,2.0,34.0,6.0,18.0,86.0,58.0,9.0,19.0,139.0,77.0,142.0,28.0,67.0,20.0
1,Collin Morikawa,9900,0.757324,23.7,46.4,16.00,100.0,4.0,1241.158,6.0,22.0,7.0,76.0,54.0,3.0,2.0,32.0,22.0,49.0,153.0,2.0,19.0,105.0
2,Justin Thomas,10100,0.736368,21.1,46.8,20.00,92.9,5.0,2043.984,8.0,100.0,5.0,21.0,17.0,1.0,84.0,2.0,36.0,67.0,55.0,111.0,43.0,114.0
3,Daniel Berger,8600,0.715518,34.2,47.5,35.00,81.0,28.0,1473.408,4.0,12.0,15.0,32.0,100.0,73.0,145.0,5.0,75.0,13.0,95.0,8.0,25.0,16.0
4,Scottie Scheffler,13300,0.699625,7.4,7.3,2.75,100.0,1.0,2800.967,1.0,2.0,1.0,63.0,23.0,4.0,30.0,1.0,1.0,7.0,68.0,26.0,3.0,2.0
5,Si Woo Kim,8300,0.698274,40.5,18.4,40.00,72.2,59.0,757.596,14.0,39.0,52.0,12.0,118.0,18.0,135.0,19.0,1.0,69.0,134.0,33.0,71.0,38.0
6,Tommy Fleetwood,9400,0.690054,18.8,20.0,20.00,100.0,12.0,1254.898,7.0,43.0,11.0,17.0,55.0,55.0,151.0,12.0,75.0,39.0,118.0,31.0,115.0,50.0
7,Keegan Bradley,8000,0.684378,27.5,54.4,40.00,92.3,18.0,797.717,5.0,23.0,13.0,19.0,148.0,63.0,135.0,19.0,58.0,18.0,65.0,45.0,55.0,105.0
8,Hideki Matsuyama,9000,0.675841,39.1,32.5,35.00,78.6,7.0,1073.294,17.0,103.0,30.0,6.0,61.0,49.0,9.0,44.0,29.0,148.0,92.0,130.0,77.0,3.0
9,Jordan Spieth,8500,0.623208,35.9,34.4,50.00,84.6,51.0,601.338,29.0,30.0,75.0,44.0,69.0,72.0,135.0,32.0,22.0,25.0,43.0,81.0,73.0,58.0


### Save to CSV

In [61]:
# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Export to CSV
filename = f"data/{tournament_config['new']['name'].replace(' ', '_')}_{tournament_config['new']['ending_date'].date()}.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/the_Memorial_Tournament_presented_by_Workday_2025-06-01.csv
